<a href="https://colab.research.google.com/github/iamtatha/POS-tagging/blob/main/FFNN/POStaggingusingNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk import word_tokenize

In [ ]:
from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000

In [ ]:
import nltk
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus

In [ ]:
print(tagged_sentences)

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]


In [ ]:
X = [] # store input sequence
Y = [] # store output sequence

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:         
        X_sentence.append(entity[0])  # entity[0] contains the word
        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag
        
    X.append(X_sentence)
    Y.append(Y_sentence)

In [ ]:
print(X[1])

['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.']


In [ ]:
from keras.preprocessing.text import Tokenizer
word_tokenizer = Tokenizer()              
word_tokenizer.fit_on_texts(X)          
X_encode = word_tokenizer.texts_to_sequences(X)  

In [ ]:
tag_token = Tokenizer()
tag_token.fit_on_texts(Y)
Y_encode = tag_token.texts_to_sequences(Y)


In [ ]:
word2idx = tag_token.word_index
# word2idx

In [ ]:
print(X[0])
print(Y[0])
print(X_encode[0])
print(Y_encode[0])

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']
[6423, 24231, 2, 7652, 102, 170, 2, 47, 1898, 1, 269, 17, 7, 13230, 619, 1711, 2761, 3]
[1, 1, 3, 11, 1, 6, 3, 2, 2, 5, 1, 4, 5, 6, 1, 1, 11, 3]


In [ ]:
lengths = [len(seq) for seq in X_encode]
print(format(max(lengths)))

271


In [ ]:
import keras

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_LENGTH=271
X_pad = pad_sequences(X_encode, maxlen=MAX_LENGTH, padding="pre", truncating="post")
Y_pad = pad_sequences(Y_encode, maxlen=MAX_LENGTH, padding="pre", truncating="post")

In [ ]:
X_final=X_pad
Y_final=Y_pad

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
path = '/content/drive/MyDrive/POStagging/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
import numpy as np

In [ ]:
EMBEDDING_SIZE  = 300  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokenizer.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

In [ ]:
print("Embeddings shape: {}".format(embedding_weights.shape))

Embeddings shape: (59449, 300)


In [ ]:
embedding_weights[word_tokenizer.word_index['pierre']]

array([ 0.07324219, -0.00994873,  0.02490234,  0.30859375,  0.10742188,
        0.20703125,  0.13085938, -0.22949219, -0.00805664, -0.02746582,
       -0.20605469, -0.3984375 , -0.04296875, -0.04370117, -0.02636719,
        0.19628906,  0.07128906,  0.18261719, -0.16992188,  0.11523438,
        0.02819824,  0.03076172,  0.171875  , -0.06225586, -0.13671875,
       -0.10888672, -0.14648438,  0.11230469, -0.05566406,  0.05712891,
        0.07617188,  0.15429688, -0.01318359, -0.02539062, -0.17578125,
        0.03979492, -0.0625    ,  0.15234375,  0.046875  , -0.19726562,
        0.09716797, -0.03857422,  0.24414062, -0.07470703,  0.15136719,
       -0.08251953, -0.16601562, -0.10107422, -0.15039062, -0.06689453,
       -0.20800781,  0.12988281,  0.10546875, -0.06689453, -0.17089844,
        0.13964844, -0.12695312, -0.02197266, -0.03100586, -0.21875   ,
        0.08642578,  0.06445312,  0.08935547,  0.02856445, -0.02685547,
       -0.23925781, -0.21484375, -0.25976562, -0.05737305,  0.25

In [ ]:
del word2vec

In [ ]:
import gc
gc.collect()

10714

In [ ]:
from keras.utils.np_utils import to_categorical
Y_final = to_categorical(Y_final)

In [ ]:
print(Y_final.shape)

(72202, 271, 13)


In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

5 fold CV

In [ ]:
from sklearn.model_selection import KFold
n_split=5

for train_index,test_index in KFold(n_split).split(X_final):
  x_train,x_test=X_final[train_index],X_final[test_index]
  y_train,y_test=Y_final[train_index],Y_final[test_index]

  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCABULARY_SIZE,EMBEDDING_SIZE,input_length=MAX_LENGTH),
    tf.keras.layers.Dense(units=Y_final.shape[2],activation='softmax')
        ])
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x_train, y_train,epochs=5,batch_size=256)
  
  print('Model evaluation ',model.evaluate(x_test,y_test))

prediction=model.predict(x_test)

Epoch 1/5
226/226 [==============================] - 34s 126ms/step - loss: 0.4907 - accuracy: 0.9803
Epoch 2/5
226/226 [==============================] - 14s 60ms/step - loss: 0.0361 - accuracy: 0.9956
Epoch 3/5
226/226 [==============================] - 10s 45ms/step - loss: 0.0206 - accuracy: 0.9963
Epoch 4/5
226/226 [==============================] - 7s 30ms/step - loss: 0.0154 - accuracy: 0.9964
Epoch 5/5
452/452 [==============================] - 2s 4ms/step - loss: 0.0259 - accuracy: 0.9922
Model evaluation  [0.025929544121026993, 0.992169976234436]
Epoch 1/5
226/226 [==============================] - 21s 91ms/step - loss: 0.4776 - accuracy: 0.9887
Epoch 2/5
226/226 [==============================] - 13s 55ms/step - loss: 0.0355 - accuracy: 0.9955
Epoch 3/5
226/226 [==============================] - 8s 36ms/step - loss: 0.0207 - accuracy: 0.9962
Epoch 4/5
226/226 [==============================] - 6s 28ms/step - loss: 0.0155 - accuracy: 0.9963
Epoch 5/5
452/452 [================

In [ ]:
model.evaluate(x_train,y_train)

Precision,F1,recall,heatmap,bleu,rouge score

In [ ]:
import sklearn.metrics

In [ ]:
prediction.shape

(14440, 271, 13)

In [ ]:
pred=[]
for i in range(prediction.shape[0]):
  l=[]
  for j in range(prediction.shape[1]):
    k=np.argmax(prediction[i][j])
    l.append(k)
  pred.append(l)


In [ ]:
np.asarray(pred).shape

(14440, 271)

In [ ]:
true=[]
for i in range(prediction.shape[0]):
  l=[]
  for j in range(prediction.shape[1]):
    k=np.argmax(prediction[i][j])
    l.append(k)
  true.append(l)

In [ ]:
np.asarray(true).shape

(14440, 271)

In [ ]:
pred=np.asarray(pred)
true=np.asarray(true)

In [ ]:
word2idx

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

In [ ]:
pred1=pred.reshape(1,pred.shape[0]*pred.shape[1])
true1=true.reshape(1,true.shape[0]*true.shape[1])

In [ ]:
print(sklearn.metrics.confusion_matrix(true1[0],pred1[0]))

[[3599833       0       0       0       0       0       0       0       0
        0       0       0       0]
 [      0   87047       0       0       0       0       0       0       0
        0       0       0       0]
 [      0       0   46607       0       0       0       0       0       0
        0       0       0       0]
 [      0       0       0   41613       0       0       0       0       0
        0       0       0       0]
 [      0       0       0       0   34523       0       0       0       0
        0       0       0       0]
 [      0       0       0       0       0   30893       0       0       0
        0       0       0       0]
 [      0       0       0       0       0       0   21261       0       0
        0       0       0       0]
 [      0       0       0       0       0       0       0   11838       0
        0       0       0       0]
 [      0       0       0       0       0       0       0       0    9061
        0       0       0       0]
 [      0       0  

In [ ]:
print(sklearn.metrics.classification_report(true1[0],pred1[0],digits=2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   3599833
           1       1.00      1.00      1.00     87047
           2       1.00      1.00      1.00     46607
           3       1.00      1.00      1.00     41613
           4       1.00      1.00      1.00     34523
           5       1.00      1.00      1.00     30893
           6       1.00      1.00      1.00     21261
           7       1.00      1.00      1.00     11838
           8       1.00      1.00      1.00      9061
           9       1.00      1.00      1.00      8609
          10       1.00      1.00      1.00     12975
          11       1.00      1.00      1.00      8878
          12       1.00      1.00      1.00       102

    accuracy                           1.00   3913240
   macro avg       1.00      1.00      1.00   3913240
weighted avg       1.00      1.00      1.00   3913240



In [ ]:
cmatrix=np.zeros((13,13))
for i in range(pred.shape[0]):
  for j in range(pred.shape[1]):
    cmatrix[true[i][j]][pred[i][j]]=cmatrix[true[i][j]][pred[i][j]]+1

In [ ]:
finalcmatrix=np.zeros((12,12))
for i in range(1,cmatrix.shape[0]):
  for j in range(1,cmatrix.shape[1]):
    finalcmatrix[i-1][j-1]=cmatrix[i][j]

In [ ]:
#confusion matrix
print(finalcmatrix)

[[87047.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.]
 [    0. 46607.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.]
 [    0.     0. 41613.     0.     0.     0.     0.     0.     0.     0.
      0.     0.]
 [    0.     0.     0. 34523.     0.     0.     0.     0.     0.     0.
      0.     0.]
 [    0.     0.     0.     0. 30893.     0.     0.     0.     0.     0.
      0.     0.]
 [    0.     0.     0.     0.     0. 21261.     0.     0.     0.     0.
      0.     0.]
 [    0.     0.     0.     0.     0.     0. 11838.     0.     0.     0.
      0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.  9061.     0.     0.
      0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.  8609.     0.
      0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0. 12975.
      0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   8878.     0.]
 [    0.     0.     0

In [ ]:
from sklearn.metrics import fbeta_score
fbeta_score(true1[0], pred1[0], average='macro', beta=2)

1.0

TESTING

In [ ]:
from nltk.tokenize import word_tokenize
def tokenize(sentence):
  oplist=word_tokenize(sentence)
  testlist=[]
  testlist.append(oplist)
  testlist
  return testlist

In [ ]:
sentence="No skipping classes from tomorrow."
testlist=tokenize(sentence)
print(testlist)

[['No', 'skipping', 'classes', 'from', 'tomorrow', '.']]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
test_samples_X = []
for s in testlist:
    s_int = []
    for w in s:
        try:
            s_int.append(word2id[w.lower()])
        except KeyError:
            pass
    test_samples_X.append(s_int)
input_length=len(test_samples_X[0]) 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='pre')
print(test_samples_X)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [ ]:
print(input_length)

6


In [ ]:
final=model.predict(test_samples_X)
sh=final.shape
final

1/1 [==============================] - 0s 24ms/step


array([[[9.9959201e-01, 3.0752137e-05, 3.3786404e-05, ...,
         3.2938151e-05, 2.8785804e-05, 3.3024968e-05],
        [9.9959201e-01, 3.0752137e-05, 3.3786404e-05, ...,
         3.2938151e-05, 2.8785804e-05, 3.3024968e-05],
        [9.9959201e-01, 3.0752137e-05, 3.3786404e-05, ...,
         3.2938151e-05, 2.8785804e-05, 3.3024968e-05],
        ...,
        [5.1463394e-05, 9.3497147e-05, 1.2227039e-04, ...,
         4.9704191e-04, 9.4878356e-05, 1.7691478e-04],
        [9.3209055e-05, 9.9852353e-01, 2.9078967e-04, ...,
         9.0474088e-05, 9.9730998e-05, 1.2870728e-04],
        [5.4564058e-05, 1.5791644e-04, 6.5237335e-05, ...,
         9.1641326e-05, 1.5154327e-04, 1.9525386e-04]]], dtype=float32)

In [ ]:
l=[]
for i in range(sh[0]):
  for j in range(sh[1]):
    l.append(np.argmax(final[i][j]))
print(l)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 1, 4, 1, 3]


In [ ]:
word2idx

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

In [ ]:
tags=[]
output=[]
tags=list(word2idx.keys())
tags
for elements in range(len(l)-input_length,len(l)):
  output.append(tags[l[elements]-1])
output

['det', 'verb', 'noun', 'adp', 'noun', '.']

plot model

In [ ]:
img_file = './model_arch.png'

tf.keras.utils.plot_model(model, to_file=img_file, show_shapes=True, show_layer_names=True)

5 FOLD CROSS VALIDATION
